# Install and Import Dependencies

In [36]:
import mediapipe as mp
import cv2

In [37]:
mp_drawing = mp.solutions.drawing_utils 
mp_holistic = mp.solutions.holistic

#  Make Detections

In [38]:
cap = cv2.VideoCapture(0)
# Mediapipe Holestic model initiation for integrating face, hand and pose components
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Webcam-Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #  Face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        #  Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        #  Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        #  Body Pose
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [39]:
# Result Landmarks
results.face_landmarks.landmark[0]

x: 0.49196335673332214
y: 0.5550404787063599
z: -0.028627831488847733

# 2. Capture Landmarks & Export to CSV
<!-- <img src="https://i.imgur.com/8bForKY.png"> -->
<!-- <img src="https://i.imgur.com/AzKNp7A.png"> -->

In [40]:
import csv
import os # helps with exporting to a folder structure
import numpy as np

In [41]:
num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark) # number of landmarks for pose (468) + number of landmarks for face (33)
num_coords

501

In [42]:
# creating Header row for CSV

landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [43]:
# landmarks

In [44]:
# Exporting Co-ordinates to CSV uding csv_writer.writerow

with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [49]:
# Exporting with a class name 

class_name = "Standing Up"

In [50]:
# Simmmilar to Part 1 but this time we export co-ordinates to CSV

cap = cv2.VideoCapture(0)
# Mediapipe Holestic model initiation for integrating face, hand and pose components
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Webcam-Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #  Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        #  Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        #  Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        #  Body Pose
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
            # Append class name 
            row.insert(0, class_name)
            
            # Export to CSV
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# Train Custom Model Using Scikit Learn

### 1 Read in Collected Data and Process

In [51]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [52]:
df = pd.read_csv('coords.csv')

In [53]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Thumbs Up,0.451005,0.482094,-1.057776,0.996735,0.476566,0.425559,-0.990926,0.994137,0.490944,...,-0.000256,0.0,0.515850,0.432699,0.021185,0.0,0.520180,0.427483,0.022160,0.0
1,Thumbs Up,0.452929,0.482317,-1.072364,0.996145,0.478736,0.426327,-0.999489,0.993117,0.492895,...,0.000601,0.0,0.516384,0.430530,0.021009,0.0,0.520783,0.425983,0.021835,0.0
2,Thumbs Up,0.453313,0.482524,-1.069538,0.995004,0.479279,0.426688,-0.994975,0.991254,0.493257,...,-0.000451,0.0,0.516779,0.427254,0.020888,0.0,0.521027,0.423301,0.021710,0.0
3,Thumbs Up,0.453257,0.482975,-1.097544,0.994895,0.479351,0.427100,-1.022864,0.991094,0.493353,...,-0.000824,0.0,0.519794,0.429573,0.021038,0.0,0.523903,0.425861,0.021910,0.0
4,Thumbs Up,0.453034,0.483928,-1.072788,0.995070,0.479321,0.427783,-1.001294,0.991361,0.493325,...,0.000027,0.0,0.520181,0.428315,0.020123,0.0,0.524406,0.424323,0.020821,0.0


In [54]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
580,Standing Up,0.598530,0.060521,-0.528224,0.999740,0.608735,0.018450,-0.489362,0.999509,0.617902,...,0.000468,0.0,0.626942,0.043207,0.019883,0.0,0.629773,0.039640,0.021022,0.0
581,Standing Up,0.608590,0.066866,-0.492849,0.999760,0.620191,0.026680,-0.463177,0.999546,0.629637,...,0.000140,0.0,0.637848,0.038535,0.021057,0.0,0.640092,0.034183,0.022352,0.0
582,Standing Up,0.621864,0.085534,-0.500725,0.999781,0.633852,0.043962,-0.470586,0.999584,0.642816,...,-0.000326,0.0,0.664616,0.054791,0.021151,0.0,0.667946,0.050125,0.022519,0.0
583,Standing Up,0.633425,0.093240,-0.536910,0.999791,0.648617,0.050918,-0.505191,0.999602,0.658792,...,0.001490,0.0,0.676467,0.067168,0.023580,0.0,0.679545,0.063619,0.025022,0.0
584,Standing Up,0.645790,0.108314,-0.505875,0.999791,0.660692,0.065406,-0.478175,0.999602,0.670381,...,-0.001599,0.0,0.694541,0.076350,0.020782,0.0,0.697341,0.071998,0.022110,0.0


In [55]:
df[df['class']=='Drowsy']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
194,Drowsy,0.418765,0.640664,-1.311260,0.997075,0.448284,0.574871,-1.290766,0.993694,0.462864,...,-0.021749,0.0,0.491390,0.585613,-0.013417,0.0,0.495796,0.574392,-0.013691,0.0
195,Drowsy,0.421457,0.636820,-1.286350,0.995841,0.451963,0.571924,-1.260503,0.991643,0.466703,...,-0.022400,0.0,0.481691,0.580215,-0.016549,0.0,0.486373,0.568828,-0.016925,0.0
196,Drowsy,0.423214,0.639003,-1.245109,0.995753,0.453288,0.572826,-1.221063,0.991409,0.467928,...,-0.022809,0.0,0.482682,0.602648,-0.016514,0.0,0.486649,0.594225,-0.017032,0.0
197,Drowsy,0.420195,0.638715,-1.283547,0.995814,0.451776,0.572346,-1.256928,0.991573,0.466936,...,-0.015901,0.0,0.485487,0.536989,-0.003153,0.0,0.489341,0.528024,-0.003291,0.0
198,Drowsy,0.420196,0.634798,-0.999864,0.995973,0.451801,0.568985,-0.976361,0.991931,0.466959,...,-0.021613,0.0,0.480616,0.592712,-0.013548,0.0,0.485013,0.582091,-0.013765,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,Drowsy,0.642854,0.695608,-0.611736,0.988619,0.662547,0.656664,-0.552718,0.982726,0.666283,...,0.002840,0.0,0.669865,0.669382,0.034624,0.0,0.673397,0.668934,0.036559,0.0
359,Drowsy,0.645503,0.695347,-0.434367,0.989453,0.663849,0.655702,-0.389359,0.984046,0.667216,...,0.001902,0.0,0.669190,0.667997,0.033686,0.0,0.672694,0.666831,0.035747,0.0
360,Drowsy,0.645803,0.695467,-0.436514,0.989902,0.663882,0.655471,-0.390667,0.984818,0.667206,...,0.001183,0.0,0.664753,0.660224,0.032833,0.0,0.668410,0.659101,0.034849,0.0
361,Drowsy,0.645109,0.696908,-0.439497,0.990063,0.662803,0.656014,-0.390615,0.985122,0.665988,...,0.002577,0.0,0.668157,0.660218,0.033368,0.0,0.671685,0.659058,0.035293,0.0


In [56]:
X = df.drop('class', axis=1) # features
y = df['class'] # target value

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [58]:
y_test

327         Drowsy
29       Thumbs Up
429    Standing Up
428    Standing Up
572    Standing Up
          ...     
234         Drowsy
427    Standing Up
391    Standing Up
319         Drowsy
6        Thumbs Up
Name: class, Length: 176, dtype: object

### 2 Train Machine Learning Classification Model

In [59]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler # for standardizing the data so that no one feture overshadows another

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [60]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [61]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [62]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [63]:
fit_models['rf'].predict(X_test)

array(['Drowsy', 'Thumbs Up', 'Standing Up', 'Standing Up', 'Standing Up',
       'Standing Up', 'Thumbs Up', 'Standing Up', 'Thumbs Up',
       'Standing Up', 'Standing Up', 'Thumbs Up', 'Thumbs Up',
       'Standing Up', 'Thumbs Up', 'Standing Up', 'Thumbs Up',
       'Thumbs Up', 'Standing Up', 'Drowsy', 'Drowsy', 'Thumbs Up',
       'Drowsy', 'Drowsy', 'Drowsy', 'Thumbs Up', 'Thumbs Up', 'Drowsy',
       'Standing Up', 'Drowsy', 'Thumbs Up', 'Drowsy', 'Thumbs Up',
       'Drowsy', 'Drowsy', 'Thumbs Up', 'Drowsy', 'Drowsy', 'Standing Up',
       'Standing Up', 'Standing Up', 'Thumbs Up', 'Drowsy', 'Thumbs Up',
       'Drowsy', 'Standing Up', 'Drowsy', 'Thumbs Up', 'Standing Up',
       'Drowsy', 'Drowsy', 'Standing Up', 'Thumbs Up', 'Standing Up',
       'Thumbs Up', 'Standing Up', 'Standing Up', 'Standing Up',
       'Standing Up', 'Standing Up', 'Drowsy', 'Thumbs Up', 'Drowsy',
       'Standing Up', 'Drowsy', 'Thumbs Up', 'Standing Up', 'Thumbs Up',
       'Standing Up', 'Standing

### 3 Evaluate and Serialize Model 

In [64]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 

In [65]:
for algo, model in fit_models.items():
    pred = model.predict(X_test)
    print(algo, accuracy_score(y_test, pred))

lr 1.0
rc 1.0
rf 1.0
gb 1.0


In [66]:
fit_models['rf'].predict(X_test)

array(['Drowsy', 'Thumbs Up', 'Standing Up', 'Standing Up', 'Standing Up',
       'Standing Up', 'Thumbs Up', 'Standing Up', 'Thumbs Up',
       'Standing Up', 'Standing Up', 'Thumbs Up', 'Thumbs Up',
       'Standing Up', 'Thumbs Up', 'Standing Up', 'Thumbs Up',
       'Thumbs Up', 'Standing Up', 'Drowsy', 'Drowsy', 'Thumbs Up',
       'Drowsy', 'Drowsy', 'Drowsy', 'Thumbs Up', 'Thumbs Up', 'Drowsy',
       'Standing Up', 'Drowsy', 'Thumbs Up', 'Drowsy', 'Thumbs Up',
       'Drowsy', 'Drowsy', 'Thumbs Up', 'Drowsy', 'Drowsy', 'Standing Up',
       'Standing Up', 'Standing Up', 'Thumbs Up', 'Drowsy', 'Thumbs Up',
       'Drowsy', 'Standing Up', 'Drowsy', 'Thumbs Up', 'Standing Up',
       'Drowsy', 'Drowsy', 'Standing Up', 'Thumbs Up', 'Standing Up',
       'Thumbs Up', 'Standing Up', 'Standing Up', 'Standing Up',
       'Standing Up', 'Standing Up', 'Drowsy', 'Thumbs Up', 'Drowsy',
       'Standing Up', 'Drowsy', 'Thumbs Up', 'Standing Up', 'Thumbs Up',
       'Standing Up', 'Standing

In [67]:
y_test

327         Drowsy
29       Thumbs Up
429    Standing Up
428    Standing Up
572    Standing Up
          ...     
234         Drowsy
427    Standing Up
391    Standing Up
319         Drowsy
6        Thumbs Up
Name: class, Length: 176, dtype: object

In [68]:
# Exporting our best Model in a file and save it

with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

#  Make Detections with Model

In [69]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [70]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [ ]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
                
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #  Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        #  Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        #  Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        #  Body Pose 
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
        

            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [72]:
tuple(np.multiply(np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)), [640,480]).astype(int))

(389, 201)